In [35]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score, make_scorer
from tqdm import tqdm

In [43]:
def bootstrap_test(metric, y, preds, null_hypothesis, n_bootstrap, seed=12345, stratified=True, alpha=95, metric_average=None):
    """
    Parameters
    ----------
    metric : fucntion
        Metric to compute, e.g. AUC for ROC curve or AP for PR curve
    y : numpy.array
        Ground truth
    preds : numpy.array
        Predictions
    null_hypothesis :
        Value for the metric if predictions are random or some other kind of null hypothesis for testing the model
        performance against
    n_bootstrap:
        Number of bootstrap samples to draw
    seed : int
        Random seed
    stratified : bool
        Whether to do a stratified bootstrapping
    alpha : float
        Confidence intervals width
    """

    np.random.seed(seed)
    metric_vals = []
    classes = np.unique(y)
    inds = []
    for cls in classes:
        inds.append(np.where(y == cls)[0])

    for _ in tqdm(range(n_bootstrap), total=n_bootstrap, desc='Bootstrap:'):
        if stratified:
            ind_bs = []
            for ind_cur in inds:
                ind_bs.append(np.random.choice(ind_cur, ind_cur.shape[0]))
            ind = np.hstack(ind_bs)
        else:
            ind = np.random.choice(y.shape[0], y.shape[0])

        if metric_average is not None:
            bootstrap_statistic = metric(y[ind], preds[ind], average=metric_average)
        else:
            bootstrap_statistic = metric(y[ind], preds[ind])

        if bootstrap_statistic is not None:
            metric_vals.append(bootstrap_statistic)

    metric_vals = np.array(metric_vals)
    p_value = np.mean(metric_vals <= null_hypothesis)   # I am still not confident on this 
                                                        # (e.g. this looks like one sided test, should it not be two-tailed? 

    if metric_average is not None:
        metric_val = metric(y, preds, average=metric_average)
    else:
        metric_val = metric(y, preds)

    ci_l = np.percentile(metric_vals, (100 - alpha) // 2)
    ci_h = np.percentile(metric_vals, alpha + (100 - alpha) // 2)

    return p_value, metric_val, ci_l, ci_h

In [49]:
def concordance_correlation_coefficient(y_true, y_pred):
    """Concordance correlation coefficient."""
    # Raw data
    dct = {
        'y_true': y_true,
        'y_pred': y_pred
    }
    df = pd.DataFrame(dct)
    # Remove NaNs
    df = df.dropna()
    # Pearson product-moment correlation coefficients
    y_true = df['y_true']
    y_pred = df['y_pred']
    cor = np.corrcoef(y_true, y_pred)[0][1]
    # Means
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    # Population variances
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    # Population standard deviations
    sd_true = np.std(y_true)
    sd_pred = np.std(y_pred)
    # Calculate CCC
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2

    return numerator / denominator

In [51]:
metrics_multiclass = {
        'Accuracy': accuracy_score,
        'Balanced Accuracy Score': balanced_accuracy_score,
        'Matthews Correlation Coefficient': matthews_corrcoef,
        'Cohen Kappa': cohen_kappa_score,
        'Lins Concordance Correlation Coefficient': concordance_correlation_coefficient,
        }

metrics_averaged = {
        'F1 Score': f1_score,
        'Precision': precision_score, 
        'Recall': recall_score,
        }

In [52]:
feature_sets = ['radiomics_dhi_dpsi', 'radiomics_only', 'dhi_dpsi_only']

dfs = []

for feature_set in feature_sets:
    predictions = pd.read_csv(f'../data/original_results/{feature_set}.csv')
    y = predictions['pfirrmann'].values
    preds = predictions['pfirrmann_prediction'].values

    evaluation_results = {}

    for name, score in metrics_multiclass.items():
        print(f'Computing {name}')
        p_value, metric_val, ci_l, ci_h = bootstrap_test(score, y, preds, null_hypothesis=0.5, n_bootstrap=1000)
        evaluation_results[name] = [metric_val, ci_l, ci_h, p_value, feature_set]

    evaluation_results = pd.DataFrame.from_dict(evaluation_results, orient='index', columns=['Metric Value', 'CI Lower', 'CI Upper', 'p-value', 'Feature Set'])
    
    for name, score in metrics_averaged.items():
        for avg in ['macro', 'micro']:
            print(f'Computing {name} ({avg})')
            p_value, metric_val, ci_l, ci_h = bootstrap_test(score, y, preds, null_hypothesis=0.5, n_bootstrap=1000, metric_average=avg)
            evaluation_results.loc[f'{name} ({avg})'] = [metric_val, ci_l, ci_h, p_value, feature_set]
    
    dfs.append(evaluation_results)

df = pd.concat(dfs, axis=0)
df.to_csv('../data/test_set_metrics.csv')

Computing Accuracy


Bootstrap:: 100%|██████████| 1000/1000 [00:00<00:00, 5309.58it/s]


Computing Balanced Accuracy Score


Bootstrap:: 100%|██████████| 1000/1000 [00:00<00:00, 1700.69it/s]


Computing Matthews Correlation Coefficient


...

Bootstrap:: 100%|██████████| 1000/1000 [00:00<00:00, 1429.81it/s]


Computing Recall (macro)


Bootstrap:: 100%|██████████| 1000/1000 [00:00<00:00, 1431.01it/s]


Computing Recall (micro)


Bootstrap:: 100%|██████████| 1000/1000 [00:00<00:00, 1421.75it/s]


In [53]:
df

,Metric Value,CI Lower,CI Upper,p-value,Feature Set
Accuracy,0.809410,0.787879,0.828573,0.00,radiomics_dhi_dpsi
Balanced Accuracy Score,0.778560,0.741511,0.811098,0.00,radiomics_dhi_dpsi
Matthews Correlation Coefficient,0.701613,0.667768,0.733129,0.00,radiomics_dhi_dpsi
Cohen Kappa,0.701291,0.667297,0.732468,0.00,radiomics_dhi_dpsi
Lins Concordance Correlation Coefficient,0.856597,0.834406,0.874548,0.00,radiomics_dhi_dpsi
F1 Score (macro),0.774959,0.742629,0.803338,0.00,radiomics_dhi_dpsi
F1 Score (micro),0.809410,0.787879,0.828573,0.00,radiomics_dhi_dpsi
Precision (macro),0.773816,0.742167,0.804680,0.00,radiomics_dhi_dpsi
Precision (micro),0.809410,0.787879,0.828573,0.00,radiomics_dhi_dpsi
Recall (macro),0.778560,0.741511,0.811098,0.00,radiomics_dhi_dpsi
